(Notebook modified from https://course.ccs.neu.edu/ds4440f20/)

# LSTMs and sequence2 sequence models

**Instructions:** Answer the questions below in the notebook itself. Submit on canvas your notebook and a pdf printout of your notebook.

## Let's work through our exercise: learning to add (with strings)

This idea borrowed from the official Keras docs -- I have borrowed some of their data generation code but written PyTorch version. See original version (keras) here: https://github.com/keras-team/keras/blob/master/examples/addition_rnn.py

Additional useful links (torch): https://github.com/bentrevett/pytorch-seq2seq


In [ ]:
import numpy as np

import torch

In [ ]:
class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one-hot integer representation
    + Decode the one-hot or integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars, one_hot=False):
        """Initialize character table.
        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
        self.one_hot = one_hot

    def encode(self, C, num_rows):
        """One-hot encode given string C.
        # Arguments
            C: string, to be encoded.
            num_rows: Number of rows in the returned one-hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        if self.one_hot:
          x = np.zeros((num_rows, len(self.chars)))
          for i, c in enumerate(C):
              x[i, self.char_indices[c]] = 1
        else:
          x = np.zeros(num_rows)
          for i, c in enumerate(C):
            x[i] = self.char_indices[c]
          
        return x

    def decode(self, x, calc_argmax=True):
        """Decode the given vector or 2D array to their character output.
        # Arguments
            x: A vector or a 2D array of probabilities or one-hot representations;
                or a vector of character indices (used with `calc_argmax=False`).
            calc_argmax: Whether to find the character index with maximum
                probability, defaults to `True`.
        """
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)

Test the encoding and decoding:

In [ ]:
import random
vocabulary = ['a', 'b', 'c', 'd', 'e']

ct = CharacterTable(vocabulary, True)

seq = ''.join([random.choice(vocabulary) for i in range(5)])
seq_enc = ct.encode(seq, 10)
print(seq)
print(seq_enc)
print(ct.decode(seq_enc))
assert seq == ct.decode(seq_enc)[:len(seq)]

print("=" * 80)
ct = CharacterTable(vocabulary, False)
seq_enc = ct.encode(seq, 10)
print(seq)
print(seq_enc)


ddded
[[0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
dddedaaaaa
ddded
[3. 3. 3. 4. 3. 0. 0. 0. 0. 0.]


Next generate data

In [ ]:
# Parameters for the model and dataset.
TRAINING_SIZE = 50000
DIGITS = 3

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS

# All the numbers, plus sign and space for padding.
chars = '0123456789+ '
ctable = CharacterTable(chars)

questions, expected = [], []
seen = set()

while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789'))
                    for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    
    # Pad the data with spaces such that it is always MAXLEN.
    q = '{}+{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    questions.append(query)
    ans = str(a + b)
    # Answers can be of maximum size DIGITS + 1.
    ans += ' ' * (DIGITS + 1 - len(ans))
    expected.append(ans)
print('Total addition questions:', len(questions))

Total addition questions: 50000


In [ ]:
# e.g.
print(list(zip(questions[:5], expected[:5])))

[('8+994  ', '1002'), ('838+732', '1570'), ('70+569 ', '639 '), ('759+162', '921 '), ('32+704 ', '736 ')]


Now vectorize, split into train and val data

In [ ]:
print('Vectorization...')
# note that the pytorch (nn) Embedding layer wants indices as inputs (not one-hot)
x = np.zeros((len(questions), MAXLEN), dtype=int) # len(chars)), dtype=int)
y = np.zeros((len(questions), DIGITS + 1), dtype=int) #, len(chars)), dtype=int)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
    
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

x[0].shape # MAXLEN x num chars

Vectorization...


(7,)

In [ ]:
x[0,:]

array([10,  1, 11, 11,  6,  0,  0])

In [ ]:
y[0].shape # (DIGITS + 1) x num chars

(4,)

In [ ]:
y[0]

array([3, 2, 2, 4])

Shuffle, split into train/val

In [ ]:
# Shuffle (x, y) in unison
indices = np.arange(len(y))
np.random.shuffle(indices)
x = torch.from_numpy(x[indices])
y = torch.from_numpy(y[indices])

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print()
print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

Training Data:
torch.Size([45000, 7])
torch.Size([45000, 4])

Validation Data:
torch.Size([5000, 7])
torch.Size([5000, 4])


Now let's define our encoder/decoder model!

In [ ]:
x[0,:]

tensor([4, 2, 3, 1, 6, 9, 2])

In [ ]:
import torch
import torch.nn as nn

class Encoder(nn.Module):
  
  def __init__(self, input_dim, emb_dim, hidden_dim):
    super().__init__()
    
    self.input_dim = input_dim
    self.embed_dim = emb_dim
    self.hidden_dim = hidden_dim
    
    self.embed_layer = nn.Embedding(self.input_dim, self.embed_dim)
    # batch_first --> (batch, seq, feature)
    self.rnn = nn.LSTM(self.embed_dim, self.hidden_dim, batch_first=True)
    
  def forward(self, x):
    # x_e = (batch x length x dims)
    x_e = self.embed_layer(x)
    outputs, h = self.rnn(x_e)
    return h
        

In [ ]:
class Decoder(nn.Module):
  
  def __init__(self, input_dim, embed_dim, hidden_dim, output_dim):
    super().__init__()
    
    self.input_dim = input_dim
    self.hidden_dim = hidden_dim
    self.embed_dim = embed_dim
    self.output_dim = output_dim
    
    self.embed_layer = nn.Embedding(self.input_dim, self.embed_dim)
    # batch_first --> (batch, seq, feature)
    self.rnn = nn.LSTM(self.embed_dim, self.hidden_dim, batch_first=True)
    self.out = nn.Linear(self.hidden_dim, self.output_dim)
    self.sm = nn.Softmax(dim=-1)
    
  
  def forward(self, x, hidden):
    x_e = self.embed_layer(x)
    output, (h, c) = self.rnn(x_e, hidden)
    out = self.out(h)
    
    y_hat = self.sm(out)
  
    return y_hat, (h, c)

**Q1. Answer the questions below:**

Read about the Encoder-Decoder Architecture here: https://d2l.ai/chapter_recurrent-modern/encoder-decoder.html

- In this particular problem, what are the elements of the input sequence (list all possible values)?
- In this particular problem, what are the elements of the output sequence (list all possible values)?
- In this particular problem, which sequence does the Encoder encode?
- Which state of the LSTM does the Encoder use as the encoding of the input sequence (recall that the LSTM calculates a state after each element of the input sequence)?

- [0,1,2,3,4,5,6,7,8,9,+,' '] are the possible values of input sequence.

- Similar to input sequence output sequence is represented with one of the numbers between 0-9 and ' ' with max length of 3.

- The encoder encodes the string sequence of adding two numbers, containing a maxing 3 digit integers.

- To Encode Input Sequence, encoder uses Hidden State of the LSTM. 

In [ ]:
import random 

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
       
    def forward(self, x, y, teacher_forcing_ratio=0.5):
        
        # src = [src sent len, batch size]
        # trg = [trg sent len, batch size]
        # teacher_forcing_ratio is probability to use teacher forcing
        # e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = y.shape[0]
        max_len = y.shape[1]
        vocab_size = self.decoder.output_dim
        
        # tensor to store decoder outputs
        outputs = torch.zeros(max_len, batch_size, vocab_size).to("cuda")
        
        # last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden = self.encoder(x)
        
        # first input to the decoder is the <sos> tokens
        y_hat = torch.zeros(batch_size, dtype=torch.long).unsqueeze(1)
        for t in range(0, max_len):
 
            output, hidden = self.decoder(y_hat, hidden)          
            outputs[t] = output
    
            teacher_force = random.random() < teacher_forcing_ratio
            # the 2 arg is the dimension to reduce; skips batch and
            # and pulls max index from each softmax (for each instance)
            max_preds = output.max(2)[1] # will be 1 x batch
            # tranpose to batch x 1
            y_hat = max_preds.transpose(0,1)
            if teacher_force:
              # then replace predictions with the reference
              y_hat = y[:,t].unsqueeze(1)

        
        # need to flip dims around to be 
        # (batch x length x vocab)
        return outputs.permute(1, 0, 2)
  

In [ ]:
y[0]

tensor([8, 9, 3, 0])

In [ ]:
embed_dim = 32
hidden_dim = 32
vocab_size = len(chars)

encoder = Encoder(vocab_size, embed_dim, hidden_dim)
decoder = Decoder(vocab_size, embed_dim, hidden_dim, vocab_size)

s2s = Seq2Seq(encoder, decoder)

output = s2s(x[:8], y[:8])

output.shape

In [ ]:
y_target = y[:8]
y_target.shape

NameError: ignored

In [ ]:
y_target[0,:]

tensor([4, 8, 2, 0])

In [ ]:
output[0,:]

tensor([[0.0838, 0.0752, 0.1075, 0.0687, 0.0741, 0.0896, 0.0731, 0.0947, 0.0818,
         0.0623, 0.1076, 0.0816],
        [0.0864, 0.0861, 0.0970, 0.0689, 0.0701, 0.0921, 0.0811, 0.0873, 0.0770,
         0.0716, 0.1010, 0.0814],
        [0.0776, 0.0818, 0.1048, 0.0703, 0.0719, 0.0899, 0.0750, 0.0838, 0.0835,
         0.0767, 0.0969, 0.0878],
        [0.0866, 0.0818, 0.1047, 0.0748, 0.0678, 0.0830, 0.0707, 0.0825, 0.0736,
         0.0781, 0.1065, 0.0900]], device='cuda:0', grad_fn=<SliceBackward>)

In [ ]:
y_target.flatten().shape

torch.Size([32])

In [ ]:
C = len(chars)

In [ ]:
# need to create (N x C) so collapse first two dims
output = output.contiguous().view(-1, C)
# this is just going to be (B x 1) -- B being batch size
y_target = y_target.flatten().cuda()

In [ ]:
output.shape

torch.Size([32, 12])

In [ ]:
from torch import optim
optimizer = optim.Adam(s2s.parameters())
criterion = nn.NLLLoss()

In [ ]:
loss = criterion(output, y_target)
print(loss.item())

-0.08508788049221039


Now, train!

In [ ]:
def train(model, x, y, optimizer, criterion, batch_size=16, epochs=10):
    model.train()
    epoch_loss = 0
    for i in range(epochs):
      current_idx = 0 
   
      while (current_idx + batch_size) < x.shape[0]:
        optimizer.zero_grad()
        batch_x, batch_y = x[current_idx:current_idx+batch_size], y[current_idx:current_idx+batch_size]
        
        output = model(batch_x, batch_y)
        
        # flatten
        #output_flat = output.contiguous().view(-1, vocab_size)
        output_flat = output.contiguous().view(-1, vocab_size)
        y_flat = batch_y.view(-1).cuda()
       
        loss = criterion(output_flat, y_flat)
       
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()

        current_idx += batch_size
        
      print(f"epoch {i} loss: {epoch_loss:.2f}")
    return model
        

In [ ]:
mm = train(s2s, x_train, y_train, optimizer, criterion, epochs=10)

epoch 0 loss: -919.32
epoch 1 loss: -1870.89
epoch 2 loss: -2825.68
epoch 3 loss: -3782.19
epoch 4 loss: -4739.48
epoch 5 loss: -5697.45
epoch 6 loss: -6655.45
epoch 7 loss: -7613.74
epoch 8 loss: -8572.94
epoch 9 loss: -9532.71


Make predictions for first 10 instances in validation set. We are passing in y_val (which we would not actually have in practice!) just for convienence -- note that we set the teacher forcing ratio to 0, and hence this is not used during decoding.



In [ ]:
predictions = s2s(x_train[:10], y_train[:10])
predictions = predictions.cpu().detach().numpy()
print(predictions.shape)
ctable.decode(predictions[3,:,:])

(10, 4, 12)


'111 '

In [ ]:
y_train0 = y_train[0].cpu().detach().numpy()
ctable.decode(y_train0, calc_argmax=False)

'260 '

In [ ]:
predictions = s2s(x_val[:10], y_val[:10], teacher_forcing_ratio=0)

In [ ]:
predictions.shape

torch.Size([10, 4, 12])

In [ ]:
predictions = predictions.cpu().detach().numpy()


In [ ]:
predictions.shape

(10, 4, 12)

In [ ]:
x_val0 = x_val[1].cpu().detach().numpy()
x_val0.shape
ctable.decode(x_val0, calc_argmax=False)

'69+937 '

In [ ]:
ctable.decode(predictions[0,:,:])

'111 '

In [ ]:
predictions[1,0,:]

array([9.6621365e-12, 1.7318649e-10, 9.0345847e-10, 1.0000000e+00,
       1.2083294e-09, 9.2383262e-10, 1.0086231e-09, 8.0573348e-10,
       1.9271300e-09, 8.5161683e-10, 1.4581296e-09, 5.4421723e-10],
      dtype=float32)

In [ ]:
y_val0 = y_val[1].cpu().detach().numpy()
ctable.decode(y_val0, calc_argmax=False)

'1006'